In [1]:
# TO DO: i just added extra nodes that will only run if recon runs in the same go. What happens to subjects 
# that have been run through recon but not the rest of the script? Or what happens after recon? Should I have a status check
# and connect them?

In [2]:
# import modules
import io, os, sys, types # needed
import glob # needed
from nipype.pipeline.engine import Workflow, Node, MapNode # needed
from nipype.interfaces.utility import Function, IdentityInterface
import nipype.interfaces.io as nio
import nipype.pipeline.engine as pe
from nipype.interfaces.freesurfer import MRIConvert
from nipype.interfaces.freesurfer import ReconAll
from nipype import config
import re
import shutil
import pathlib

In [3]:
def scan(subject):
    subsessions = glob.glob(dicomdir+subject+'/Accelerated_Sagittal_MPRAGE/*/*/')
    repeat_tag = '-'
    for num in range(len(subsessions)):
        # look to see if more than one session on the same date
        # then look to see if more than one date (or both)
        parentfolder = subsessions[num].split('/')[8]
        filename = os.listdir(subsessions[num])[0] # dicom name to extract date
        date = re.search('raw_'+r'+[0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9]', filename).group()[4:]
        if num == len(subsessions)-1:
            namedate = dicomdir+subject+'_'+date
            os.rename(dicomdir+subject, namedate)
        else:
            namedateplus = dicomdir+subject+'_'+date+repeat_tag+'/Accelerated_Sagittal_MPRAGE/'+folder1
            pathlib.Path(namedateplus).mkdir(parents=True, exist_ok=True)
            shutil.move(subsessions[num], namedateplus)
            if not os.listdir(dicomdir+subject+'/Accelerated_Sagittal_MPRAGE/'+parentfolder):
                os.rmdir(dicomdir+subject+'/Accelerated_Sagittal_MPRAGE/'+parentfolder)
            repeat_tag = repeat_tag+'-'

In [4]:
# specify variables
leadsdir = '/cluster/animal/scan_data/leads/'
os.chdir(leadsdir)
dicomdir = "/cluster/animal/scan_data/leads/LEADS/"
unpacklog = "/autofs/cluster/animal/scan_data/leads/recon/unpack.log"
recondir = '/autofs/cluster/animal/scan_data/leads/recon_nip/'
folders = [x for x in os.listdir(dicomdir) if not x.startswith(".")]
subjlist = [f for f in os.listdir(dicomdir) if (("_") not in f) and (not f.startswith('.') and ("duplicate" not in f))]

# wipe clean batch.recon.list
open(recondir+'batch.recon.list', 'w').close()

# unwravel all subjects with multiple sessions and rename to include date
for sub in subjlist:
    scan(sub)

# now just make a list of subjects by ID (this will be the input to the nodes)
# will define dicom path within node
sh_dicomlist = [f for f in os.listdir(dicomdir) if (("_" in f) and ("REPEAT_RUNS" not in f))]

# define workflow
leads_workflow = Workflow(name='leads_workflow') #, base_dir = '/autofs/cluster/animal/scan_data/leads/recon_nip/') # add  base_dir='/shared_fs/nipype_scratch'

# configure to stop on first crash
cfg = dict(execution={'stop_on_first_crash': True})
config.update_config(cfg)

In [5]:
# for debugging
sh_dicomlist = ['LDS0370007_20180801']


In [6]:
# TEST NODE : PASSSWORDS

def credentials(): # combined with find_dicom
    import getpass
    USER = getpass.getuser()
    print('Please enter your PASSWORD for launchpad access: ')
    PASS= getpass.getpass()
    return USER, PASS

PASSWORDS = pe.Node(Function(input_names=["user", "pw"],
                         output_names=["USER","PASS"], # actual dicom (redundant to create unpacking node visualization)
                         function=credentials),
                        name='PASSWORDS')


In [7]:
# NODE : CREATEDIR
def createdir(val, USER, PASS):
    import os
    import re
    import glob
    val = val.split('/')[-1]
    dicomdir = "/autofs/cluster/animal/scan_data/leads/LEADS/"
    recondir = '/autofs/cluster/animal/scan_data/leads/recon_nip/' 
    reconpath = recondir+val+'/'
    imgpath = reconpath+'/mri/orig/'
    dumplocation = imgpath+'001.mgz'
    subject = val.split('_')[0]
    date = val.split('_')[1].replace("-","")  # take out any '-'
    MPRAGE_path = glob.glob(dicomdir+val+'/Accelerated_Sagittal_MPRAGE/*/*')[0]
    pickdicom = glob.glob(dicomdir+val+'/Accelerated_Sagittal_MPRAGE/*/*/*')[0]
    sessionid = MPRAGE_path.split("/")[-1]
    try:
        os.makedirs(imgpath)
    except(FileExistsError):
        pass
    return reconpath, MPRAGE_path, pickdicom, dumplocation, recondir, USER, PASS
        
CREATEDIR = pe.Node(Function(input_names=["val", "USER", "PASS"],
                         output_names=["createdir_out1","createdir_out2", "createdir_out3", "createdir_out4", "createdir_out5", "USER", "PASS"], # actual dicom (redundant to create unpacking node visualization)
                         function=createdir),
                        name='CREATEDIR')

In [8]:
di = '/autofs/cluster/animal/scan_data/leads/recon_nip/LDS0370007_20180801/'
subname = di.split('/')[-1]
print(subname)

In [9]:
# NODE : UNPACK

def unpack(subjectdir, MPRAGE_path):
    from os import system
    import csv
    import pandas as pd
    import os.path
    if not os.path.isfile(subjectdir+'scan.info'): 
        cmdstring = 'unpacksdcmdir -src %s -targ %s -scanonly %s/scan.info' % (MPRAGE_path, subjectdir, subjectdir)
        system(cmdstring)
    if not os.path.isfile(subjectdir+'scaninfo.csv'):
        with open(subjectdir+'/scan.info', 'r') as in_file:
            for line in in_file:
                editline = line.split()
                with open(subjectdir+'/scaninfo.csv', 'w') as result:
                    wr = csv.writer(result, dialect='excel')
                    wr.writerow(editline)
                result.close()
            in_file.close()
    scan_info = pd.read_csv(subjectdir+'/scaninfo.csv', header=None)
    subname = subjectdir.split('/')[-2]
    #try creating a txt file with subname
    with open('/autofs/cluster/animal/scan_data/leads/recon_nip/SUBNAME_'+subname, 'w') as f:
        f.write('subjectdir variable is '+subjectdir)
        f.write('subname variable is '+subname)
    return subname, subjectdir, scan_info

UNPACK = pe.Node(Function(input_names=["subjectdir","MPRAGE_path"],
                         output_names=["unpack_out1","unpack_out2", "unpack_out3"], # actual dicom (redundant to create unpacking node visualization)
                         function=unpack),
                        name='UNPACK')


In [10]:
# NODE : CONVERT2MGZ
# I think I can set subject as variable to pass on here and others send elsewhere

CONVERT2MGZ = pe.Node(MRIConvert(out_type='mgz'), 
                        name='CONVERT2MGZ')

In [11]:
# NODE SCAN_AND_LOG
# note: decided to add this afterward precaution to increase efficiency because there are few errors
# and want to run the unpack and convert2mgz in parallel)

def scan_and_log(subjectdir, scan_info, mgz, reconfolder, subname):
    import re
    import os
    import pandas as pd
    # load in the scaninfo file
    dicomdir = "/cluster/animal/scan_data/leads/LEADS/"
    check = scan_info.iloc[0,2] # first row (only row); second col (validity)
    if check != 'ok':
        with open(reconfolder+'/scanerrors', "a") as efile:
            efile.write(scan_info.iloc[0,7]) # log for errors in dicoms (or any ommitted scans)
            # delete entire recon folder
    else:
        with open(reconfolder+'/batch.recon.list', "a") as bfile:
            bfile.write(subname)
        with open(reconfolder+'/unpack.log', "a") as ufile:
            ufile.write(subname)
        # should I makea scannotes? (will add info after recon)
        Elements = {'scan_notes': [''],'loni_overallpass': [''], 'download_date':[''], 'xnat_upload':[''], 'recon_path':[''],'recon_notes':[''], 'dickerson_overallpass':['']}
        df = pd.DataFrame(Elements, columns= ['scan_notes', 'loni_overallpass', 'download_date','xnat_upload','recon_path','recon_notes','dickerson_overallpass'])
        df.to_csv(subjectdir+'/scannotes.csv')
    return subjectdir, subname

SCAN_AND_LOG = pe.Node(Function(input_names=["subjectdir","scan_info",'mgz', 'reconfolder', 'subname'],
                         output_names=["subjectdir", "subname"],
                         function=scan_and_log),
                        name='SCAN_AND_LOG')

In [13]:
# NODE RECON_JOB

def recon_job(subjectname, USER, PASS): # add in username, pass, and subjectname
    from paramiko import SSHClient
    host="launchpad"
    user=USER
    pw=PASS
    client=SSHClient()
    client.load_system_host_keys()
    client.connect(host,username=user,password=pw, look_for_keys=False)
    tmpstr = '(cd /autofs/cluster/animal/scan_data/leads/analyses_nip; setenv p %s ; ./batch.recon.sh)' % subjectname
    stdin, stdout, stderr = client.exec_command(tmpstr)
    #stin = print("stdin: ", stdin.readlines())
    err = "stderr: ", stderr.readlines()
    out = "pwd: ", stdout.readlines()
    if len(err) < 1:
        warning = '0'
    else:
        warning = '1'
    return err, out, warning, subjectname

RECON_JOB = pe.Node(Function(input_names=["subjectname","USER", "PASS"], 
                        output_names=[ 'err', 'out', 'warning','subjectname'],
                         function=recon_job),
                        name='RECON_JOB')


In [14]:
# NODE: GATHER_FS_DETAILS (this part only after recon is done)

def gather_FS_details(subjectname): # add in username, pass, and subjectname
    recondir = '/autofs/cluster/animal/scan_data/leads/recon_nip/'
    try:
        # obtain FS version
        versionfile = open(recondir+subjectname+'/scripts/build-stamp.txt', 'r')
        versionstring = versionfile.read()
        version = versionstring.split('-')
        result = [i for i in version if i.startswith('v')][0]
        long = result[1:]
        
        #obtain short verison of long
        size = len(long)
        x = 0
        while x ==0:
            if (long[-1] == '0') or (long[-1] == '.'): # shave off any . or 0s from the end of version number.
                long = long[:-1]
            else:
                x =1
        vlabel = 'FS'+long

        # obtain run number
        with open(recondir+subjectname+'/scaninfo.csv','r') as f:
            reader = csv.reader(f)
            scan_list = list(reader)
            runstring = scan_list[0][0] # run
            if len(runstring) == 1:
                runstring = '0'+runstring
        recon_name = vlabel+'_'+runstring

    except(FileNotFoundError):
        print(subjectname+" freesurfer recon not complete.")
        recon_name = 'FS--'
    return subjectname, recon_name
        
FS_DETAILS = pe.Node(Function(input_names=["subjectname"], 
                        output_names=[ 'subject', 'recon_name'],
                         function=gather_FS_details),
                        name='FS_DETAILS')

In [15]:
# NODE : MAKE_ORIG_FOLDER

def create_orig_folder(subjectname, recon_name):
    recondir = '/autofs/cluster/animal/scan_data/leads/recon_nip/'
    freesurfer_dirs = ['mri', 'stats', 'tmp', 'trash', 'touch', 'label', 'surf', 'scripts']
    # now go into each subject folder and create a folder with recon_name
    try:
        os.mkdir(recondir+subjectname+'/'+recon_name)
    except(FileExistsError):
        print(subjectname+" original recon folder already created.")
    
    # move all subfolders into this recon_name folder
    for fsdir in freesurfer_dirs:
        FS = Path(recondir+subjectname+'/'+fsdir)
        if FS.exists():
            shutil.move(recondir+subjectname+'/'+fsdir, recondir+subjectname+'/'+recon_name+'/'+fsdir)
    return subjectname, recon_name, recondir

MAKE_ORIGINAL_DIR = pe.Node(Function(input_names=["subjectname", "recon_name"], 
                        output_names=['subjectname', 'recon_name', 'recondir'],
                         function=create_orig_folder),
                        name='MAKE_ORIGINAL_DIR')



In [16]:
# NODE : PREPARE_MANEDITS

def preparing_manedits(subjectname, recon_name, recondir):
    analysesdir = '/autofs/cluster/animal/scan_data/leads/analyses_nip/'
    try:
        recon_name2 = 'unedit.'+recon_name
        FS2 = Path(recondir+subjectname+'/'+recon_name2)
        shutil.copytree(recondir+subjectname+'/'+recon_name, FS2) # if edit.recon folder does not already exist, add to log
        with open(recondir+'to_edit.list\n', "a") as myfile:
            myfile.write(sub)
    except(FileExistsError):
        print(subjectname+" unedit.recon folder already created.")
        
    #Now create symlink to edit. in analysis folder!!
    try:
        os.symlink(recondir+subjectname+'/'+recon_name2, analysesdir+sub)
    except(FileExistsError):
        print(subjectname+" in analyses is already linked to unedit.recon.")
    return subjectname, recon_name, recondir
        
PREPARE_MANEDITS = pe.Node(Function(input_names=['subjectname', 'recon_name', 'recondir'], 
                        output_names=['subjectname', 'recon_name', 'recondir'],
                         function=preparing_manedits),
                        name='PREPARE_MANEDITS')

In [17]:
# # # NODE : INFOSOURCE
INFOSOURCE = Node(IdentityInterface(fields=['subject_name'], mandatory_inputs=False),
                  name="INFOSOURCE")

INFOSOURCE.iterables = ('subject_name', sh_dicomlist)

# NODE : SELECTFILES
#templates = dict(dicom=sh_dicomlist[0])    ## THIS WORKED!
templates = {
    "dicom": "{subject_name}" 
    }
SELECTFILES = Node(nio.SelectFiles(templates, base_directory=dicomdir),
                   name="SELECTFILES")

# NODE : DATASINK
DATASINK = Node(nio.DataSink(base_directory=leadsdir,
                container='recon_nip'),
                name="DATASINK")

In [18]:
# Connect all nodes (including INFOSOURCE, SELECTFILES, and DATASINK) to workflow

leads_workflow.connect([(INFOSOURCE, SELECTFILES, [('subject_name', 'subject_name')]),
                (SELECTFILES, CREATEDIR, [('dicom', 'val')]),
                (PASSWORDS, CREATEDIR, [('USER', 'USER')]),
                (PASSWORDS, CREATEDIR, [('PASS', 'PASS')]), 
                (CREATEDIR, UNPACK, [('createdir_out1', 'subjectdir')]),
                 (CREATEDIR, UNPACK, [('createdir_out2', 'MPRAGE_path')]),
                 (CREATEDIR, CONVERT2MGZ, [('createdir_out3', 'in_file')]),
                 (CREATEDIR, CONVERT2MGZ, [('createdir_out4', 'out_file')]),   
                (CONVERT2MGZ, SCAN_AND_LOG, [('out_file', 'mgz')]),
                (CREATEDIR, SCAN_AND_LOG, [('createdir_out5', 'reconfolder')]),
                (UNPACK, SCAN_AND_LOG, [('unpack_out1', 'subname')]),
                (UNPACK, SCAN_AND_LOG, [('unpack_out2', 'subjectdir')]),
                (UNPACK, SCAN_AND_LOG, [('unpack_out3', 'scan_info')]),
                (CREATEDIR, RECON_JOB, [('USER','USER')]),     # new
                (CREATEDIR, RECON_JOB, [('PASS','PASS')]),     # new
                (SCAN_AND_LOG, RECON_JOB, [('subname','subjectname')]), 
                (RECON_JOB, DATASINK, [('warning','backup')])  # backup folder?
#                 (RECON_JOB, FS_DETAILS, [('subjectname','subjectname')]),
#                 (FS_DETAILS, MAKE_ORIGINAL_DIR, [('subject','subjectname')]), 
#                 (FS_DETAILS, MAKE_ORIGINAL_DIR, [('recon_name','recon_name')]), 
#                 (MAKE_ORIGINAL_DIR, PREPARE_MANEDITS, [('subjectname','subjectname')]), 
#                 (MAKE_ORIGINAL_DIR, PREPARE_MANEDITS, [('recon_name','recon_name')]), 
#                 (MAKE_ORIGINAL_DIR, PREPARE_MANEDITS, [('recondir','recondir')]), 
#                 (PREPARE_MANEDITS, DATASINK, [('subjectname','backup')])  # backup folder?
                 ])

In [19]:
# Execute your workflow in sequential way
# leads_workflow.run(run(plugin='MultiProc', plugin_args={'n_procs' : 2})
leads_workflow.run()

leads_workflow.write_graph(graph2use='flat')

190315-13:12:48,194 nipype.workflow INFO:
	 Workflow leads_workflow settings: ['check', 'execution', 'logging', 'monitoring']
190315-13:12:48,230 nipype.workflow INFO:
	 Running serially.
190315-13:12:48,232 nipype.workflow INFO:
	 [Node] Setting-up "leads_workflow.SELECTFILES" in "/tmp/tmp8jp_z26a/leads_workflow/_subject_name_LDS0370007_20180801/SELECTFILES".
190315-13:12:48,236 nipype.workflow INFO:
	 [Node] Running "SELECTFILES" ("nipype.interfaces.io.SelectFiles")
190315-13:12:48,246 nipype.workflow INFO:
	 [Node] Finished "leads_workflow.SELECTFILES".
190315-13:12:48,248 nipype.workflow INFO:
	 [Node] Setting-up "leads_workflow.PASSWORDS" in "/tmp/tmp9t5spt71/leads_workflow/PASSWORDS".
190315-13:12:48,251 nipype.workflow INFO:
	 [Node] Running "PASSWORDS" ("nipype.interfaces.utility.wrappers.Function")
Please enter your PASSWORD for launchpad access: 
········
190315-13:12:56,760 nipype.workflow INFO:
	 [Node] Finished "leads_workflow.PASSWORDS".
190315-13:12:56,762 nipype.workflo

190315-13:13:18,332 nipype.interface INFO:
	 stdout 2019-03-15T13:13:18.309812:	NImageCols        240
190315-13:13:18,333 nipype.interface INFO:
	 stdout 2019-03-15T13:13:18.309812:	NFrames           1
190315-13:13:18,334 nipype.interface INFO:
	 stdout 2019-03-15T13:13:18.309812:	SliceArraylSize   1
190315-13:13:18,334 nipype.interface INFO:
	 stdout 2019-03-15T13:13:18.309812:	IsMosaic          0
190315-13:13:18,335 nipype.interface INFO:
	 stdout 2019-03-15T13:13:18.309812:	ImgPos             98.7542 146.4407 122.8136 
190315-13:13:18,336 nipype.interface INFO:
	 stdout 2019-03-15T13:13:18.309812:	VolRes              1.0000   1.0000   1.0000 
190315-13:13:18,336 nipype.interface INFO:
	 stdout 2019-03-15T13:13:18.309812:	VolDim            240      256      208 
190315-13:13:18,337 nipype.interface INFO:
	 stdout 2019-03-15T13:13:18.309812:	Vc                 -0.0000  -1.0000   0.0000 
190315-13:13:18,337 nipype.interface INFO:
	 stdout 2019-03-15T13:13:18.309812:	Vr                 

'/autofs/cluster/animal/scan_data/leads/graph.png'

In [20]:
leads_workflow.write_graph(graph2use='flat')

190315-13:13:55,496 nipype.workflow INFO:
	 Generated workflow graph: /autofs/cluster/animal/scan_data/leads/graph.png (graph2use=flat, simple_form=True).


'/autofs/cluster/animal/scan_data/leads/graph.png'